In [1]:
import pandas as pd
import numpy as np
import os
import sys

sys.path.append(os.path.join(os.path.dirname(os.getcwd())))
from interfaces.alchemy import AlchemyInterface
import configparser
from db.schema import StationLocation, Station, Messages
from geopy.distance import great_circle
from datetime import datetime
import re
from http.cookiejar import CookieJar
import requests
from bs4 import BeautifulSoup
from random import choice

In [2]:
c_parser = configparser.ConfigParser()
c_parser.read("../../config.ini")
config = {"db": dict(c_parser["db"])}

In [17]:
target = "EA3GKP"

In [18]:
alchemy_interface = AlchemyInterface(config)

In [19]:
data_station = alchemy_interface.select_obj(Station, **{"station_id": target})

data_location = alchemy_interface.select_obj(
    StationLocation,
    ["sync_id", "timestamp", "latitude", "longitude", "country"],
    df=True,
    **{"station": target},
)
data_messages = alchemy_interface.select_obj(
    Messages,
    ["sync_id", "dst_station", "path", "timestamp", "comment"],
    df=True,
    **{"src_station": target},
)

In [20]:
pd.merge(data_location, data_messages, on="sync_id", how="right").dropna(
    subset=["latitude", "longitude", "timestamp_y"]
).sort_values(by="timestamp_x").tail(10).drop("sync_id", axis=1)

,timestamp_x,latitude,longitude,country,dst_station,path,timestamp_y,comment
48,2024-05-07 22:30:27.061016,41.37367,2.10450,Spain,APLG01,"{TCPIP*,qAC,T2LAUSITZ}",2024-05-07 22:30:27.061016,Barcelona LoRa iGATE Station
49,2024-05-07 22:30:27.776109,41.39567,2.13950,Spain,APDW15,"{TCPIP*,qAC,T2SWEDEN}",2024-05-07 22:30:27.776109,--->igate Barcelona
50,2024-05-07 22:30:35.212644,28.48233,-16.25633,NaN,APDW15,"{TCPIP*,qAC,T2CSNGRAD}",2024-05-07 22:30:35.212644,--->igate Santa Cruz de Tenerife
52,2024-05-07 22:30:35.865678,28.46617,-16.27167,NaN,APTW01,"{WIDE3-3,qAO,EA3GKP-12}",2024-05-07 22:30:35.865678,Meteo Santa Cruz de Tenerife 130m aSl (Ultimet...
53,2024-05-07 22:30:36.094951,41.33000,2.08933,Spain,APLG01,"{TCPIP*,qAC,T2SPAIN}",2024-05-07 22:30:36.094951,Barcelona-El Prat LoRa iGATE Station
55,2024-05-07 22:30:44.164192,28.46617,-16.27167,NaN,APTW01,"{WIDE3-3,qAO,EA3GKP-12}",2024-05-07 22:30:44.164192,Meteo Santa Cruz de Tenerife 130m aSl (Ultimet...
56,2024-05-07 22:30:44.247839,41.33000,2.08933,Spain,APLG01,"{TCPIP*,qAC,T2UKRAINE}",2024-05-07 22:30:44.247839,Barcelona-El Prat LoRa iGATE Station
59,2024-05-07 22:30:53.160685,28.48233,-16.25633,NaN,APDW15,"{TCPIP*,qAC,T2CSNGRAD}",2024-05-07 22:30:53.160685,--->igate Santa Cruz de Tenerife
60,2024-05-07 22:30:53.212349,28.46617,-16.27167,NaN,APTW01,"{WIDE3-3,qAO,EA3GKP-12}",2024-05-07 22:30:53.212349,Meteo Santa Cruz de Tenerife 130m aSl (Ultimet...
61,2024-05-07 22:30:53.608758,41.33000,2.08933,Spain,APLG01,"{TCPIP*,qAC,T2SPAIN}",2024-05-07 22:30:53.608758,Barcelona-El Prat LoRa iGATE Station


In [11]:
packets = pd.merge(data_messages, data_location, on="sync_id", how="left")

In [23]:
print([(index < 3 or index > 9) if True else False for index in range(0, 9)])

[True, True, True, False, False, False, False, False, False]


In [ ]:
alchemy_interface.search_text(table=Messages, language="English", text="station")

In [ ]:
class QRZ:
    def __init__(self):
        config = configparser.ConfigParser()
        config.read("../../config.ini")
        self.account_usage = {}
        for section in config.sections():
            if section.startswith("qrz_"):
                self.account_usage[
                    (config[section]["username"], config[section]["password"])
                ] = 0
        if len(self.account_usage.items()) == 0:
            raise ValueError("Config file does not contain any valid 'qrz' account")
        self.base_url = "https://www.qrz.com/"
        jar = CookieJar()
        self.session = requests.Session()
        self.session.cookies = jar
        self.is_logged = False

    def prettify(self, text):
        return text.lower().replace(" ", "_").replace("?", "").replace("#", "")

    def format_date(self, date, format):
        date = datetime.strptime(date, format)
        return date.isoformat()

    def roll_accounts(self):
        # choose a random username and password that has an account usage of less than 25
        choices = [item[0] for item in self.account_usage.items() if item[1] < 25]
        if not choices:
            return None
        login = choice(choices)
        self.account_usage[login] += 1
        return login

    def login(self):
        self.login_data = self.roll_accounts()
        if not self.login_data:
            return None
        response = self.session.post(
            "https://www.qrz.com/login",
            data={"username": self.login_data[0], "password": self.login_data[1]},
        )
        if response.status_code == 200:
            self.is_logged = True
        return True

    def get_station(self, station):
        if not self.login():
            print("Daily limit reached for all accounts")
            return None
        print("Requesting page")
        self.response = self.session.get(self.base_url + "db/" + station)
        station_info = {}
        if self.response.status_code != 200:
            print("Could not fetch page: ", self.response.status_code)
            return None
        if "Too many lookups" in self.response.text:
            print("Daily limit reached")
            self.account_usage[self.login_data] = 25
            return self.get_station(station)

        soup = BeautifulSoup(self.response.content, "html.parser")
        station_info["name"] = " ".join(
            soup.find_all("span", {"style": "color: black; font-weight: bold"})[0]
            .getText()
            .split()
        )

        station_info["img"] = (
            soup.find("div", id="calldata").find("img", id="mypic").get("src")
        )
        station_info["biography"] = soup.find("divalue", id="biodata")
        rows = []
        for _, row in enumerate(soup.find("table", id="detbox").find_all("tr")):
            row_content = [el.text.strip() for el in row.find_all("td")]
            if row_content:
                rows.append(row_content)
        table_data = rows[1:]

        geo = {}
        for item in table_data:
            if item[0].lower() in [
                "longitude",
                "grid_square",
                "geo source",
                "latitude",
            ]:
                geo[self.prettify(item[0])] = item[1]
            elif item[0].lower() == "othercallsigns":
                table_alias = soup.find("th", string="Alias").parent.parent.parent
                aliases = {}
                for row in table_alias.find_all("tr"):
                    if row.find_all("td"):
                        alias = row.find_all("td")[0]
                        aliases[alias.text] = (
                            alias.find("a")["href"] if alias.find("a") else None
                        )
                if aliases:
                    station_info["alias"] = aliases
                continue
            else:
                if len(item) > 1:
                    station_info[self.prettify(item[0])] = item[1]
        if geo.get("longitude") and geo.get("latitude"):
            geo["longitude"] = float(geo["longitude"].split()[0])
            geo["latitude"] = float(geo["latitude"].split()[0])

        geo["address"] = ", ".join(
            list(soup.find_all("p", {"class": "m0"})[0].stripped_strings)[
                4 if station_info.get("nickname") else 2 :
            ]
        )
        station_info["geo"] = geo

        for extra_field in ["qsl_by_mail", "uses_lotw", "qsl_by_eqsl"]:
            if extra_field in station_info:
                station_info[extra_field] = station_info[extra_field][:3].strip()

        qrz_data = {}
        for _, (field, value) in enumerate(station_info.items()):
            try:
                if not re.search(
                    "[a-zA-Z0-9]{1,3}[0-9][a-zA-Z0-9]{0,3}[a-zA-Z]", field
                ):
                    qrz_data[field] = value
            except Exception:
                print("Error parsing field: ", field)

        qrz_data["date_joined"] = self.format_date(
            qrz_data["date_joined"], "%Y-%m-%d %H:%M:%S"
        )
        qrz_data["last_update"] = self.format_date(
            qrz_data["last_update"], "%Y-%m-%d %H:%M:%S"
        )

        return qrz_data

In [ ]:
qrz = QRZ()

In [ ]:
station_data = qrz.get_station(target)

In [ ]:
data = station_data.copy()

In [ ]:
data

In [ ]:
# data.pop("alias")
# data.pop("geo")
data

In [ ]:
pd.Series(data).iterrows()

In [ ]:
station_info = alchemy_interface.select_obj(
    Station, "*", df=True, **{"station_id": target}
)
station_locations = alchemy_interface.select_obj(
    StationLocation, "*", df=True, **{"station": target}
)
station_messages_src = alchemy_interface.select_obj(
    Messages, "*", df=True, **{"src_station": target}
)
station_messages_dst = alchemy_interface.select_obj(
    Messages, "*", df=True, **{"dst_station": target}
)

In [ ]:
station_info.ssid.describe()

In [ ]:
def analyze_timestamps(timestamps):
    # Convertir los timestamps a un objeto DateTime
    timestamps = pd.to_datetime(timestamps)

    # Calcular la frecuencia de los timestamps
    time_diffs = timestamps.diff()  # Diferencia entre cada timestamp
    median_freq = time_diffs.median()
    std_diff = time_diffs.std()

    # Reajustar el índice de time_diffs para alinear con timestamps
    time_diffs = time_diffs.reindex(timestamps.index, method="ffill")

    max_accepted_gap = median_freq + std_diff * 2
    gaps = timestamps[time_diffs > max_accepted_gap]

    # Información adicional
    mean_freq = time_diffs.mean()
    min_freq = time_diffs.min()
    max_freq = time_diffs.max()
    start_date = timestamps.min()
    end_date = timestamps.max()
    num_timestamps = len(timestamps)
    total_duration = end_date - start_date

    # Crear un diccionario con los resultados
    analysis_results = {
        "mean_frequency": mean_freq,
        "median_frequency": median_freq,
        "min_frequency": min_freq,
        "max_frequency": max_freq,
        "start_date": start_date,
        "end_date": end_date,
        "gaps": gaps,
        "num_timestamps": num_timestamps,
        "recorded_time": total_duration,
    }

    return analysis_results

In [ ]:
analyze_timestamps(station_locations["timestamp"])

In [ ]:
def analyze_locations(locations):
    # Convertir las ubicaciones a un formato numpy para un fácil manejo
    locations_np = np.array(locations)

    # Calcular la distancia total y la distancia promedio entre puntos
    distances = [
        great_circle(locations_np[i], locations_np[i + 1]).kilometers
        for i in range(len(locations_np) - 1)
    ]
    total_distance = sum(distances)
    average_distance = np.mean(distances) if distances else 0
    std_dev_distance = np.std(distances) if distances else 0

    # Función para calcular el punto medio
    def calculate_midpoint(locs):
        lat = locs[:, 0]
        lon = locs[:, 1]
        return np.mean(lat), np.mean(lon)

    # Calcular el punto medio
    midpoint = calculate_midpoint(locations_np)

    # Encontrar los extremos geográficos
    northernmost = max(locations, key=lambda x: x[0])
    southernmost = min(locations, key=lambda x: x[0])
    easternmost = max(locations, key=lambda x: x[1])
    westernmost = min(locations, key=lambda x: x[1])

    # Calcular la distancia máxima desde el punto medio
    max_distance_from_mid = max(
        great_circle(midpoint, loc).kilometers for loc in locations
    )

    # Calcular el área aproximada cubierta por los puntos
    latitudes = locations_np[:, 0]
    longitudes = locations_np[:, 1]
    approx_area = (
        great_circle(
            (latitudes.min(), longitudes.min()), (latitudes.min(), longitudes.max())
        ).kilometers
        * great_circle(
            (latitudes.min(), longitudes.min()), (latitudes.max(), longitudes.min())
        ).kilometers
    )

    return {
        "locations": locations,
        "total_distance_km": total_distance,
        "average_distance_km": average_distance,
        "std_dev_distance_km": std_dev_distance,
        "midpoint": midpoint,
        "northernmost": northernmost,
        "southernmost": southernmost,
        "easternmost": easternmost,
        "westernmost": westernmost,
        "max_distance_from_midpoint_km": max_distance_from_mid,
        "approximate_area_sq_km": approx_area,
    }

In [ ]:
loc = analyze_locations(station_locations[["latitude", "longitude"]].values.tolist())

In [ ]:
import plotly.graph_objects as go


def plot_map(analysis):
    # Extraer información del análisis
    midpoint = analysis["midpoint"]
    northernmost = analysis["northernmost"]
    southernmost = analysis["southernmost"]
    easternmost = analysis["easternmost"]
    westernmost = analysis["westernmost"]

    # Coordenadas para la bounding box
    lon_box = [
        westernmost[1],
        easternmost[1],
        easternmost[1],
        westernmost[1],
        westernmost[1],
    ]
    lat_box = [
        northernmost[0],
        northernmost[0],
        southernmost[0],
        southernmost[0],
        northernmost[0],
    ]

    # Crear figura
    fig = go.Figure()

    # Añadir bounding box como un camino cerrado
    fig.add_trace(
        go.Scattermapbox(
            lon=lon_box,
            lat=lat_box,
            mode="lines",
            line=go.scattermapbox.Line(color="grey"),
            name="Bounding Box",
        )
    )

    # Añadir puntos extremos y punto medio
    for point, name in zip(
        [northernmost, southernmost, easternmost, westernmost, midpoint],
        ["Northernmost", "Southernmost", "Easternmost", "Westernmost", "Midpoint"],
    ):
        fig.add_trace(
            go.Scattermapbox(
                lon=[point[1]],
                lat=[point[0]],
                mode="markers",
                marker=go.scattermapbox.Marker(size=5, color="black"),
            )
        )

    # Establecer la vista inicial del mapa
    fig.update_layout(
        mapbox_style="open-street-map",
        mapbox={"center": {"lon": midpoint[1], "lat": midpoint[0]}, "zoom": 10},
    )
    lats, lons = zip(*analysis["locations"])
    fig.add_trace(
        go.Scattermapbox(
            lon=lons,
            lat=lats,
            mode="markers+lines",
            line=go.scattermapbox.Line(color="black"),
            marker=go.scattermapbox.Marker(size=10, color="blue"),
            name="Bounding Box",
        )
    )

    # Mostrar figura
    fig.show()

In [ ]:
def analyze_loc_temporal(locations_timestamp):
    # Convert column to datetime
    locations_timestamp["timestamp"].apply(lambda x: pd.to_datetime(x))
    coordinates = list(
        zip(locations_timestamp["latitude"], locations_timestamp["longitude"])
    )
    locations_timestamp = locations_timestamp.assign(coordinate=coordinates)
    locations_timestamp["time_elapsed"] = (
        locations_timestamp.groupby("coordinate")["timestamp"]
        .diff()
        .fillna(pd.Timedelta(seconds=0))
    )

    # Calcular el tiempo total pasado en cada coordenada
    total_time_elapsed = locations_timestamp.groupby("coordinate")["time_elapsed"].sum()

    # Calcular la frecuencia de visitas a cada coordenada
    visit_frequency = locations_timestamp["coordinate"].value_counts()

    # Calcular la fecha y hora de la primera y última visita a cada coordenada
    first_visit = locations_timestamp.groupby("coordinate")["timestamp"].min()
    last_visit = locations_timestamp.groupby("coordinate")["timestamp"].max()

    # Calcular la duración total de la estancia en cada coordenada
    total_stay_duration = last_visit - first_visit

    # Crear un DataFrame con los resultados
    results_df = pd.DataFrame(
        {
            "total_time_elapsed": total_time_elapsed,
            "visit_frequency": visit_frequency,
            "first_visit": first_visit,
            "last_visit": last_visit,
            "total_stay_duration": total_stay_duration,
        }
    )

    return results_df

In [ ]:
analyze_loc_temporal(station_locations[["latitude", "longitude", "timestamp"]])

In [ ]:
def analyze_comment(comments):
    # Calcular la frecuencia de comentarios distintos
    unique_comments_freq = comments.value_counts()

    pattern = re.compile(
        r"\b((https?|ftp):\/\/)?([\w-]+(\.[\w-]+)+)([\/\w-]*)*(\?\w+=\w+(&\w+=\w+)*)?\b"
    )
    urls = comments.apply(lambda x: pattern.search(x).group(0))
    has_url = urls.apply(lambda x: x is not None)

    results_list = []
    for comment, freq, has_url_val, url in zip(
        unique_comments_freq.index, unique_comments_freq.values, has_url, urls
    ):
        result_dict = {
            "Comment": comment,
            "Freq": freq,
            "has_url": has_url_val,
            "URL": url,
        }
        results_list.append(result_dict)

    return results_list

In [ ]:
analyze_comment(station_messages_src["comment"])